In [1]:
from bs4 import BeautifulSoup
import urllib
import sys  
from PyQt4.QtGui import *  
from PyQt4.QtCore import *  
from PyQt4.QtWebKit import *  
from lxml import html 
import dryscrape

In [ ]:
session = dryscrape.Session()
session.visit(my_url)
response = session.body()
soup = BeautifulSoup(response)

In [2]:
#class for rendering js pages for use with BS
class Render(QWebPage):  
    def __init__(self, url):  
        self.app = QApplication(sys.argv)  
        QWebPage.__init__(self)  
        self.loadFinished.connect(self._loadFinished)  
        self.mainFrame().load(QUrl(url))  
        self.app.exec_()  
  
    def _loadFinished(self, result):  
        self.frame = self.mainFrame()  
        self.app.quit() 

In [2]:
#list of all mathces in list links
def season_links(year):
    resultsPage = urllib.urlopen("http://www.fourfourtwo.com/statszone/results/8-" + year)
    resultsPageSoup = BeautifulSoup(resultsPage)
    scores = resultsPageSoup.findAll('td', {'class': 'score'})
    homeTeams = resultsPageSoup.findAll('td', {'class': 'home-team'})
    awayTeams = resultsPageSoup.findAll('td', {'class': 'away-team'})
    #matches = [homeTeams[i].contents[0].strip()+"_"+awayTeams[i].contents[0].strip() for i in range(0,len(homeTeams))]
    gamesStatsLinks = ['http://www.fourfourtwo.com' + 
                       l.a['href'] for l in resultsPageSoup.findAll('td', {'class': 'link-to-match'})]

    season = {}
    
    for i in range(0,len(homeTeams)):
        season[homeTeams[i].contents[0].strip() + "_" + awayTeams[i].contents[0].strip()] = {
            "homeTeam":homeTeams[i].contents[0],
            "awayTeam":awayTeams[i].contents[0],
            "homeGoals":int(scores[i].contents[0].split("-")[0]),
            "awayGoals":int(scores[i].contents[0].split("-")[1]),
            "link":gamesStatsLinks[i]
        }
    
    return(season)

In [9]:
scores = resultsPageSoup.findAll('td', {'class': 'score'})

In [10]:
scores

[]

In [5]:
resultsPage = urllib.urlopen("http://www.fourfourtwo.com/statszone/results/8-" + "2015")

In [8]:
resultsPageSoup

<!DOCTYPE html>\n<!--[if IEMobile 7]><html class="iem7"  lang="en" dir="ltr"><![endif]--><!--[if lte IE 6]><html class="lt-ie9 lt-ie8 lt-ie7"  lang="en" dir="ltr"><![endif]--><!--[if (IE 7)&(!IEMobile)]><html class="lt-ie9 lt-ie8"  lang="en" dir="ltr"><![endif]--><!--[if IE 8]><html class="lt-ie9"  lang="en" dir="ltr"><![endif]--><!--[if (gte IE 9)|(gt IEMobile 7)]><!--><html dir="ltr" lang="en"><!--<![endif]-->\n<head>\n<meta charset="unicode-escape"/>\n<meta content="on" http-equiv="x-dns-prefetch-control"/>\n<link href="//images.cdn.fourfourtwo.com" rel="dns-prefetch"/>\n<link href="//css.cdn.fourfourtwo.com" rel="dns-prefetch"/>\n<link href="//cdn.fourfourtwo.com" rel="dns-prefetch"/>\n<!--[if IE 9]>\n<link rel="prefetch" href="//images.cdn.fourfourtwo.com" />\n<link rel="prefetch" href="//css.cdn.fourfourtwo.com" />\n<link rel="prefetch" href="//cdn.fourfourtwo.com" />\n<![endif]-->\n<link href="http://cdn.fourfourtwo.com/sites/fourfourtwo.com/themes/fourfourtwo/favicon.ico" rel="

In [7]:
resultsPageSoup = BeautifulSoup(resultsPage)

In [3]:
test = season_links("2015")
test

/home/lucas/anaconda2/envs/football/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /home/lucas/anaconda2/envs/football/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


{}

In [83]:
import json
test = season_links("2015")
test
with open('out.json', 'w') as fp:
    json.dump(test, fp)

TypeError: key (u'West Bromwich Albion', u'Sunderland') is not a string

In [34]:
def match_stats(link):
    resultsPage = urllib.urlopen(link)
    resultsPageSoup = BeautifulSoup(resultsPage)

    #overall statistics
    match_stats = {}
    possession = resultsPageSoup.findAll('div', {'id':'summary_possessions'})[0].findAll('svg', {'class': 'doughnut_chart'})[0].text
    match_stats['home_possession'] = float(possession.split('%')[0])
    match_stats['away_possession'] = float(possession.split('%')[1])
    
    
    match_stats['home_pass_cmplt'] = int(resultsPageSoup.findAll('div', {'id':'summary_passes'})[0].findAll('text', {'fill': "#FCD800"})[0].next)
    match_stats['away_pass_cmplt'] = int(resultsPageSoup.findAll('div', {'id':'summary_passes'})[0].findAll('text', {'fill': "#E6E6E6"})[0].next)

    match_stats['home_crnr'] = int(resultsPageSoup.findAll('div', {'id':'summary_corners'})[0].findAll('text', {'fill': "#FCD800"})[0].next)
    match_stats['away_crnr'] = int(resultsPageSoup.findAll('div', {'id':'summary_corners'})[0].findAll('text', {'fill': "#E6E6E6"})[0].next)

    match_stats['home_attck_pass'] = int(resultsPageSoup.findAll('div', {'id':'summary_attacking'})[0].findAll('text', {'fill': "#FCD800"})[0].next)
    match_stats['away_attck_pass']= int(resultsPageSoup.findAll('div', {'id':'summary_attacking'})[0].findAll('text', {'fill': "#E6E6E6"})[0].next)

    match_stats['home_shots'] = int(resultsPageSoup.findAll('div', {'id':'summary_shots'})[0].findAll('text', {'fill': "#FCD800"})[0].next)
    match_stats['away_shots'] = int(resultsPageSoup.findAll('div', {'id':'summary_shots'})[0].findAll('text', {'fill': "#E6E6E6"})[0].next)

    match_stats['home_fouls'] = int(resultsPageSoup.findAll('div', {'id':'summary_fouls'})[0].findAll('text', {'fill': "#FCD800"})[0].next)
    match_stats['away_fouls'] = int(resultsPageSoup.findAll('div', {'id':'summary_fouls'})[0].findAll('text', {'fill': "#E6E6E6"})[0].next)

    match_stats['date_location'] = resultsPageSoup.findAll('div', {'class': 'teams'})[0].next.strip()
    
    match_stats['date'] = match_stats['date_location'].split(',')[-2]
    match_stats['time'] = match_stats['date_location'].split('-')[-1]
    return(match_stats)


In [35]:
stats_test = match_stats(links[1])

In [36]:
stats_test

{'away_attck_pass': 86,
 'away_crnr': 4,
 'away_fouls': 6,
 'away_pass_cmplt': 334,
 'away_possession': 40.4,
 'away_shots': 5,
 'date': u' May 15',
 'date_location': u'Emirates Stadium London, Sunday, May 15, 2016 - 15:00',
 'home_attck_pass': 283,
 'home_crnr': 5,
 'home_fouls': 12,
 'home_pass_cmplt': 537,
 'home_possession': 59.6,
 'home_shots': 16,
 'time': u' 15:00'}

In [ ]:
posession

In [42]:
#this is updated to output home and away team dictionaries
#[ToDo] copy over to the updated class

def player_links(game_link):
    url = game_link + '/player-stats#tabs-wrapper-anchor'

    
    session = dryscrape.Session()
    session.visit(url)
    formatted_result = session.body()
    

    soup = BeautifulSoup(formatted_result, 'html5')
    home_players = soup.findAll('div', {'class': 'lineup home'})
    away_players = soup.findAll('div', {'class': 'lineup away'})


    home_player_urls = ['http://www.fourfourtwo.com' + player.findAll('a')[0]['href'] 
       for player in home_players]
    home_player_names = [player.findAll('a')[0].contents
       for player in home_players]

    away_player_urls = ['http://www.fourfourtwo.com' + player.findAll('a')[0]['href'] 
       for player in away_players]
    away_player_names = [player.findAll('a')[0].contents
       for player in away_players]

    home_players = {}
    for i in range(len(home_player_names)):
        home_players[home_player_names[i][0]] = {"link":home_player_urls[i] }

    away_players = {}
    for i in range(len(away_player_names)):
        away_players[away_player_names[i][0]] = {"link":away_player_urls[i] }
    
    return(home_players, away_players)


In [44]:
home, away = player_links('http://www.fourfourtwo.com/statszone/8-2015/matches/803532')

In [46]:
away

{u'Ayew': {'link': u'http://www.fourfourtwo.com/statszone/8-2015/matches/803532/player-stats/80146/OVERALL_02#tabs-wrapper-anchor'},
 u'Bacuna': {'link': u'http://www.fourfourtwo.com/statszone/8-2015/matches/803532/player-stats/74297/OVERALL_02#tabs-wrapper-anchor'},
 u'Bunn': {'link': u'http://www.fourfourtwo.com/statszone/8-2015/matches/803532/player-stats/10954/OVERALL_02#tabs-wrapper-anchor'},
 u'Cissokho': {'link': u'http://www.fourfourtwo.com/statszone/8-2015/matches/803532/player-stats/44354/OVERALL_02#tabs-wrapper-anchor'},
 u'Gueye': {'link': u'http://www.fourfourtwo.com/statszone/8-2015/matches/803532/player-stats/80801/OVERALL_02#tabs-wrapper-anchor'},
 u'Lescott': {'link': u'http://www.fourfourtwo.com/statszone/8-2015/matches/803532/player-stats/7551/OVERALL_02#tabs-wrapper-anchor'},
 u'Lyden': {'link': u'http://www.fourfourtwo.com/statszone/8-2015/matches/803532/player-stats/160220/OVERALL_02#tabs-wrapper-anchor'},
 u'Sinclair': {'link': u'http://www.fourfourtwo.com/statsz

In [40]:
game_link = 'http://www.fourfourtwo.com/statszone/8-2015/matches/803532'
url = game_link + '/player-stats#tabs-wrapper-anchor'


session = dryscrape.Session()
session.visit(url)
formatted_result = session.body()

soup = BeautifulSoup(formatted_result, 'html5')
home_players = soup.findAll('div', {'class': 'lineup home'})
away_players = soup.findAll('div', {'class': 'lineup away'})


home_player_urls = ['http://www.fourfourtwo.com' + player.findAll('a')[0]['href'] 
   for player in home_players]
home_player_names = [player.findAll('a')[0].contents
   for player in home_players]

away_player_urls = ['http://www.fourfourtwo.com' + player.findAll('a')[0]['href'] 
   for player in away_players]
away_player_names = [player.findAll('a')[0].contents
   for player in away_players]
    
home_players = {}
for i in range(len(home_player_names)):
    home_players[home_player_names[i][0]] = {"link":home_player_urls[i] }
        
away_players = {}
for i in range(len(away_player_names)):
    away_players[away_player_names[i][0]] = {"link":away_player_urls[i] }

In [41]:
away_players

{u'Ayew': {'link': u'http://www.fourfourtwo.com/statszone/8-2015/matches/803532/player-stats/80146/OVERALL_02#tabs-wrapper-anchor'},
 u'Bacuna': {'link': u'http://www.fourfourtwo.com/statszone/8-2015/matches/803532/player-stats/74297/OVERALL_02#tabs-wrapper-anchor'},
 u'Bunn': {'link': u'http://www.fourfourtwo.com/statszone/8-2015/matches/803532/player-stats/10954/OVERALL_02#tabs-wrapper-anchor'},
 u'Cissokho': {'link': u'http://www.fourfourtwo.com/statszone/8-2015/matches/803532/player-stats/44354/OVERALL_02#tabs-wrapper-anchor'},
 u'Gueye': {'link': u'http://www.fourfourtwo.com/statszone/8-2015/matches/803532/player-stats/80801/OVERALL_02#tabs-wrapper-anchor'},
 u'Lescott': {'link': u'http://www.fourfourtwo.com/statszone/8-2015/matches/803532/player-stats/7551/OVERALL_02#tabs-wrapper-anchor'},
 u'Lyden': {'link': u'http://www.fourfourtwo.com/statszone/8-2015/matches/803532/player-stats/160220/OVERALL_02#tabs-wrapper-anchor'},
 u'Sinclair': {'link': u'http://www.fourfourtwo.com/statsz

In [37]:
away_player_names[0][0]


u'Bunn'

u'/statszone/8-2015/matches/803532/player-stats/11334/OVERALL_02#tabs-wrapper-anchor'

In [13]:
players = [for a in home_players]

11

In [7]:
players = player_links('http://www.fourfourtwo.com/statszone/8-2015/matches/803532')

/home/lucas/anaconda2/envs/football/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /home/lucas/anaconda2/envs/football/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html5lib")

  markup_type=markup_type))


In [8]:
players

{u'Ayew': {'link': u'http://www.fourfourtwo.com/statszone/8-2015/matches/803532/player-stats/80146/OVERALL_02#tabs-wrapper-anchor'},
 u'Bacuna': {'link': u'http://www.fourfourtwo.com/statszone/8-2015/matches/803532/player-stats/74297/OVERALL_02#tabs-wrapper-anchor'},
 u'Beller\xedn': {'link': u'http://www.fourfourtwo.com/statszone/8-2015/matches/803532/player-stats/98745/OVERALL_02#tabs-wrapper-anchor'},
 u'Bunn': {'link': u'http://www.fourfourtwo.com/statszone/8-2015/matches/803532/player-stats/10954/OVERALL_02#tabs-wrapper-anchor'},
 u'Cazorla': {'link': u'http://www.fourfourtwo.com/statszone/8-2015/matches/803532/player-stats/19524/OVERALL_02#tabs-wrapper-anchor'},
 u'Cech': {'link': u'http://www.fourfourtwo.com/statszone/8-2015/matches/803532/player-stats/11334/OVERALL_02#tabs-wrapper-anchor'},
 u'Cissokho': {'link': u'http://www.fourfourtwo.com/statszone/8-2015/matches/803532/player-stats/44354/OVERALL_02#tabs-wrapper-anchor'},
 u'Coquelin': {'link': u'http://www.fourfourtwo.com/s

In [5]:
def player_stats(url):
    #
    playerPage = Render(url)
    playerPageRendered = playerPage.frame.toHtml()
    playerPageFormatted = str(playerPageRendered.toAscii())
    playerSoup = BeautifulSoup(playerPageFormatted, 'html5')
    
    match_stats = {}
    #passes
    match_stats['pass_success'] = len(playerSoup.findAll('line', {'marker-end': 'url(#smallblue)'}))
    match_stats['pass_fail'] = len(playerSoup.findAll('line', {'marker-end': 'url(#smallred)'}))
    match_stats['chances'] = len(playerSoup.findAll('line', {'marker-end': 'url(#smalldeepskyblue)'}))
    match_stats['assists'] = len(playerSoup.findAll('line', {'marker-end': 'url(#smallyellow)'}))

    #shots
    match_stats['on_target'] = len(playerSoup.findAll('line', {'marker-end': 'url(#bigblueend)'}))
    match_stats['off_target'] = len(playerSoup.findAll('line', {'marker-end': 'url(#bigredend)'}))
    match_stats['on_target'] = len(playerSoup.findAll('line', {'marker-end': 'url(#bigblueend)'}))
    match_stats['goals'] = len(playerSoup.findAll('line', {'marker-start': 'url(#bigyellowend)'}))

    #takeons
    match_stats['takeon_success'] = len(playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/success.png'}))
    match_stats['takeon_fail'] = len(playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/fail.png'}))

    #defensive
    match_stats['fail_tkl'] = len(playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/failed_tackle.png'}))
    match_stats['success_tkl'] = len(playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/successful_tackle.png'}))
    match_stats['success_clearence'] = len(playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/successful_clearence.png'}))
    match_stats['failed_clearence'] = len(playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/failed_clearence.png'}))
    match_stats['interception'] = len(playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/interceptions.png'}))
    match_stats['ball_recovery'] = len(playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/defensive-ball-clearence.png'}))
    match_stats['blocks'] = len(playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/blocks.png'}))
    match_stats['blocks_cross'] = len(playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/blocks-cross.png'}))

    #aerial
    match_stats['aerial_won'] = len(playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/won.png'}))
    match_stats['aerial_lost'] = len(playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/lost.png'}))

    #fouls
    match_stats['foul_cmt'] = len(playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/commited.png'}))
    match_stats['foul_sfr'] = len(playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/suffered.png'}))

    #errors
    match_stats['error_goal'] = len(playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/error-leading-goal.png'}))
    match_stats['error_shot'] = len(playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/error-leading-shot.png'}))
    
    return(match_stats)
    
    
    

In [ ]:
url = players['Gueye']['link']

In [6]:
stats = player_stats('http://www.fourfourtwo.com/statszone/8-2015/matches/803532/player-stats/80801/OVERALL_02#tabs-wrapper-anchor')

/home/lucas/anaconda2/envs/football/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /home/lucas/anaconda2/envs/football/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html5lib")

  markup_type=markup_type))


In [7]:
stats

{'aerial_lost': 0,
 'aerial_won': 0,
 'assists': 0,
 'ball_recovery': 0,
 'blocks': 0,
 'blocks_cross': 0,
 'chances': 1,
 'error_goal': 0,
 'error_shot': 0,
 'fail_tkl': 0,
 'failed_clearence': 0,
 'foul_cmt': 2,
 'foul_sfr': 1,
 'goals': 0,
 'interception': 1,
 'off_target': 0,
 'on_target': 0,
 'pass_fail': 7,
 'pass_success': 53,
 'success_clearence': 0,
 'success_tkl': 2,
 'takeon_fail': 2,
 'takeon_success': 2}

In [10]:
len(stats['aerial_lost'])

0

In [6]:
playerPage = Render('http://www.fourfourtwo.com/statszone/8-2015/matches/803532/player-stats/80801/OVERALL_02#tabs-wrapper-anchor')
playerPageRendered = playerPage.frame.toHtml()
playerPageFormatted = str(playerPageRendered.toAscii())
playerSoup = BeautifulSoup(playerPageFormatted, 'html5')

/home/lucas/anaconda2/envs/football/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /home/lucas/anaconda2/envs/football/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html5lib")

  markup_type=markup_type))


In [ ]:

#
playerPage = Render(url)
playerPageRendered = playerPage.frame.toHtml()
playerPageFormatted = str(playerPageRendered.toAscii())
playerSoup = BeautifulSoup(playerPageFormatted, 'html5')

match_stats = {}
#passes
match_stats['pass_success'] = playerSoup.findAll('line', {'marker-end': 'url(#smallblue)'})
match_stats['pass_fail'] = playerSoup.findAll('line', {'marker-end': 'url(#smallred)'})
match_stats['chances'] = playerSoup.findAll('line', {'marker-end': 'url(#smalldeepskyblue)'})
match_stats['assists'] = playerSoup.findAll('line', {'marker-end': 'url(#smallyellow)'})

#shots
match_stats['on_target'] = playerSoup.findAll('line', {'marker-end': 'url(#bigblueend)'})
match_stats['off_target'] = playerSoup.findAll('line', {'marker-end': 'url(#bigredend)'})
match_stats['on_target'] = playerSoup.findAll('line', {'marker-end': 'url(#bigblueend)'})
match_stats['goals'] = playerSoup.findAll('line', {'marker-start': 'url(#bigyellowend)'})

#takeons
match_stats['takeon_success'] = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/success.png'})
match_stats['takeon_fail'] = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/fail.png'})

#defensive
match_stats['fail_tkl'] = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/failed_tackle.png'})
match_stats['success_tkl'] = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/successful_tackle.png'})
match_stats['success_clearence'] = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/successful_clearence.png'})
match_stats['failed_clearence'] = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/failed_clearence.png'})
match_stats['interception'] = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/interceptions.png'})
match_stats['ball_recovery'] = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/defensive-ball-clearence.png'})
match_stats['blocks'] = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/blocks.png'})
match_stats['blocks_cross'] = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/blocks-cross.png'})

#aerial
match_stats['aerial_won'] = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/won.png'})
match_stats['aerial_lost'] = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/lost.png'})

#fouls
match_stats['foul_cmt'] = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/commited.png'})
match_stats['foul_sfr'] = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/suffered.png'})

#errors
match_stats['error_goal'] = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/error-leading-goal.png'})
match_stats['error_shot'] = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/error-leading-shot.png'})





In [7]:
players.keys()[1]

u'Gueye'

In [ ]:
stats = player_stats(players['Gueye']['link'])

In [3]:
url = 'http://www.fourfourtwo.com/statszone/8-2015/matches/803532/player-stats/37265/OVERALL_02#tabs-wrapper-anchor'
playerPage = Render(url)
playerPageRendered = playerPage.frame.toHtml()
playerPageFormatted = str(playerPageRendered.toAscii())
playerSoup = BeautifulSoup(playerPageFormatted, 'html5')


/Users/lucaslafone/anaconda/envs/MLlib/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html5lib")

  markup_type=markup_type))


In [5]:
soup.findAll('line', {'marker-start': 'url(#bigyellowend)'})

[<line class="pitch-object timer-1-4" marker-end="url(#bigyellow)" marker-start="url(#bigyellowend)" style="stroke:yellow;stroke-width:3" x1="621.16" x2="692" y1="258.095" y2="246.139"></line>,
 <line class="pitch-object timer-2-77" marker-end="url(#bigyellow)" marker-start="url(#bigyellowend)" style="stroke:yellow;stroke-width:3" x1="658.512" x2="692" y1="230.34" y2="264.073"></line>,
 <line class="pitch-object timer-2-79" marker-end="url(#bigyellow)" marker-start="url(#bigyellowend)" style="stroke:yellow;stroke-width:3" x1="643.7" x2="692" y1="234.183" y2="255.96"></line>]

In [4]:
soup.findAll('image', {'xlink:href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/success.png'})

[]

In [15]:
len(chances)

4

In [5]:

#passes
pass_success = playerSoup.findAll('line', {'marker-end': 'url(#smallblue)'})
pass_fail = playerSoup.findAll('line', {'marker-end': 'url(#smallred)'})
chances = playerSoup.findAll('line', {'marker-end': 'url(#smalldeepskyblue)'})
assists = playerSoup.findAll('line', {'marker-end': 'url(#smallyellow)'})

#shots
on_target = playerSoup.findAll('line', {'marker-end': 'url(#bigblueend)'})
off_target = playerSoup.findAll('line', {'marker-end': 'url(#bigredend)'})
on_target = playerSoup.findAll('line', {'marker-end': 'url(#bigblueend)'})
goals = playerSoup.findAll('line', {'marker-start': 'url(#bigyellowend)'})

#takeons
takeon_success = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/success.png'})
takeon_fail = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/fail.png'})

#defensive
fail_tkl = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/failed_tackle.png'})
success_tkl = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/successful_tackle.png'})
success_clearence = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/successful_clearence.png'})
failed_clearence = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/failed_clearence.png'})
interception = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/interceptions.png'})
ball_recovery = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/defensive-ball-clearence.png'})
blocks = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/blocks.png'})
blocks_cross = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/blocks-cross.png'})

#aerial
aerial_won = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/won.png'})
aerial_lost = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/lost.png'})

#fouls
foul_cmt = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/commited.png'})
foul_sfr = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/suffered.png'})

#errors
error_goal = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/error-leading-goal.png'})
error_shot = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/error-leading-shot.png'})


In [7]:
len(goals)
len(pass_success)

42

[<image height="529" width="740" x="0" xlink:href="/sites/fourfourtwo.com/modules/custom/statzone/files/statszone_football_pitch.png" y="0"></image>]